<font color='red' size='5px'/> Data Preprocessing<font/>

<font color='blue' size='5px'/> Introduction<font/>

#  Overview 

We have decleared the dataset to be [Dataset](https://iplab.dmi.unict.it/Deepfakechallenge/). Now we want to choose how to perform preprocessing on this dataset. We've decided that we are going to split into two teams each team will be working with different methedology. The first will use Keras and the other will use Albumentations libarary

We want to use Data augmentation techniques to increase the size and diversity of a dataset by applying transformations to the existing data. This helps to prevent overfitting and improves the generalization of the model.

#  Literature Review

## 1 The Face Deepfake Detection Challenge

### 1.1 Overview

1. Data Collection: The authors collected a large dataset of real and fake face images from various sources, including publicly available datasets and images generated using deepfake techniques.

2. Data Preparation: The dataset was split into training, validation, and testing sets. The authors also applied various data augmentation techniques to increase the size of the training set and improve the generalization of the model.

3. Baseline Model: The authors developed a baseline deep learning model for face detection using a combination of convolutional neural networks (CNNs) and recurrent neural networks (RNNs). The model was trained on the training set and evaluated on the validation set.

4. Challenge: The authors organized a challenge in which participants were asked to develop their own deepfake detection models and submit their results for evaluation. The challenge provided a platform for researchers to compare their methods and identify the most effective techniques.

5. Evaluation: The authors evaluated the performance of the participating models using various metrics, including precision, recall, and F1 score. They also compared the performance of the top-performing models with the baseline model.

### 1.2 Data Augmentation Techniques

1. Horizontal flipping:
    - Horizontally flipped some of the images in the dataset to create mirror images. 
    - This increases the diversity of the training set and makes the model more robust to variations in facial orientation.

2. Random cropping:
    - Cropped some of the images in the dataset to create new images with different sizes and aspect ratios.
    - To recognize partially obscured or have different proportions.

3. Gaussian noise: 
    - The authors added random Gaussian noise to some of the images in the dataset to simulate noise that might be present in real-world images. 
    - This helps the model to learn to recognize faces in noisy or low-quality images.

4. Random brightness and contrast: 
    - Randomly adjusted the brightness and contrast of some of the images in the dataset to simulate variations in lighting conditions. 
    - This helps the model to learn to recognize faces in different lighting conditions.
    -  For example, the brightness can be adjusted by adding a random value between -50 and 50 to the pixel values, while the contrast can be adjusted by multiplying the pixel values by a random value between 0.5 and 1.5.

### 1.3 Different Values used
- size= (128,128)
- Batch size= 64
- Data Augmentation parameters:
  - Image compression: images were compressed with the JPEG algorithm at a quality factor picked uniformaly in the range[50:99]
  - Noise: Images were corrupted with additive Gaussian noise with variable limit in ragne[10,50]
  - Blurring: Gaussian blurring was applied to the images, with blur a limit of 3 and sigma limit of 0

## 2 Investigating the Impact of Pre-processing and Prediction Aggregation on the Deepfake Detection Task

## 2.1 Overview

The paper focuses on exploring the impact of pre-processing and prediction aggregation techniques on the deepfake detection task. 

The authors use a state-of-the-art deep learning model called XceptionNet for the detection task and evaluate the model's performance on three publicly available datasets:
1. Celeb-DF,
2. DeepFake-TIMIT,
3. FaceForensics++.

## 2.2 Preprocessing Techniques

He studied the performance change due to:
- Grayscale conversion,
- Resizing,
- Histogram equalization,
- median filtering

He found out:
- grayscale conversion and histogram equalization significantly improve the detection performance on all three datasets,
- while resizing and median filtering have a limited impact or even worsen the performance.

## 2.3 Aggregation Techniques

The authors also explore three prediction aggregation techniques, namely majority voting, averaging, and stacking, to combine the predictions of multiple models or multiple runs of the same model to further enhance the accuracy.

He found out:
- The results show that majority voting and averaging are effective and efficient techniques for prediction aggregation,
- while stacking has a higher computational cost and may not always improve the performanc

# 3 An Improved Dense CNN Architecture for Deepfake Image Detection

## 3.1 Overview

<font color='blue' size='5px'/> Meetings<font/>

# 1 Conclusion

We conducted a meeting to discuss parameters used in each of the papers and the conclusions werwe:
- Normalization will not performed in this phase, but we will make it in training.
- We will need to perform image compression to reduce the size during training.
- Do small blurring, as we already applied image compression. 
- We will use MTCNN for face detection, which will improve our performance
- We have 2 parts in Data some generated by GAN, and others are real, so we will augment real data by value 1 to 3 and fake data by value 1 to 6
- We will split data into train, test, val by random.

# 2 Parameters Used

- Both teams (keras and albumentations team) will work accordingly to create augmented images of each folder (that is CelebA, FFHQ, ATTGAN,...)  separately.
- Few augmentation parameters to be followed:
  - Resize images to = (160, 160) (must be same for both teams)
  - Random rotation (0 to 360)
  - Horizontal or Vertical flip
  - Shear Range(0.2), zoom range(0.2) (can vary for both teams)
  - Height or width shift by 0.2 (can vary)
  - Blackout of either eyes, face or mouth (very imp)
  - Image compression (lower limit = 50, upper limit = 100)
  - Gaussian Blurring or Gaussian Noise (either of both or together also can be used, just make sure the image is not degraded or blurred a lot!) 
  - People working on Deepfake image folder, keep value of blurring and noise less as images are already blurred! Just have a look at images properly before applying Blur or Noise
  - Either Random brightness, saturation, hue contrast (comes under ColorJitter in albumentations)
  - Avoid cropping or even if you do, use only central crop and that too with low probability. (to avoid information loss)
- (Using too many augmentations can even degrade the dataset!) 
Don’t Normalise/rescale images right now. This will be done later while model training.
- Exploit features like OneOf provided in Albumentations library.
(link → Composition API (core.composition) - Albumentations Documentation )

- Reference :

  1.[Overview and visualization of pixel-level transforms from albumentations package|Data-science-blog](https://tugot17.github.io/data-science-blog/albumentations/data-augmentation/tutorial/2020/09/20/Pixel-level-transforms-using-albumentations-package.html) 
      
  2.[Index - Albumentations Documentation](https://albumentations.ai/docs/api_reference/augmentations/)
  
  3.Very Important read before setting values of probabilities and before using OneOf :

  [Setting probabilities for transforms - Albumentations Documentation ](https://albumentations.ai/docs/getting_started/setting_probabilities/)


<font color='blue' size='5px'/> Implement in Project<font/>

# 1 Packages

In [1]:
!pip install facenet-pytorch
!pip install -U albumentations
from PIL import Image
from facenet_pytorch.models.mtcnn import MTCNN
from google.colab.patches import cv2_imshow
from random import randint
import random
import zipfile
import cv2
import dlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 KB 1.2 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import albumentations as A
import os

# 2 Explore Data

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
celebA_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/0-CelebA"
ffhq_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/0-FFHQ"
attgan_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/1-ATTGAN"
gdwct_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/1-GDWCT"
stylegan_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/1-STYLEGAN"
stylegan2_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/1-STYLEGAN2/1-STYLEGAN2"
stargan_path = "/content/drive/MyDrive/Omdena DeepFake Image Detection (preprocessed data)/Train/1-StarGAN"

# 3 Freature Engineering 

## 3.1 Black Out function

In [4]:
def blackout_eyes(img, boxes, landmarks):
  left_eye_x, left_eye_y = landmarks[0][0][0], landmarks[0][0][1]
  right_eye_x, right_eye_y = landmarks[0][1][0], landmarks[0][1][1]

  box_width, box_height = boxes[0][2]*0.8, boxes[0][3]*0.1              
  #defining width and height of black rectangle to be created
  width_margin, height_margin = 0.1*box_width, box_height*0.4
  ## old--->> width_margin, height_margin = abs(left_eye_x - right_eye_x)*0.25, 10
  #width_margin, height_margin = width*0.8, box_height*0.1
  if(landmarks[0][0][1] >= landmarks[0][1][1]):
    start = (math.ceil(left_eye_x - width_margin), math.ceil(left_eye_y - height_margin))
    end = (math.ceil(right_eye_x + width_margin), math.ceil(right_eye_y + height_margin))
  else:
    end = (math.ceil(left_eye_x - width_margin), math.ceil(left_eye_y - height_margin))
    start = (math.ceil(right_eye_x + width_margin), math.ceil(right_eye_y + height_margin))
  cv2.rectangle(img, start, end, (0,0,0),-1)
  return img

def blackout_nose(img, boxes, landmarks):
  nose_x, nose_y = landmarks[0][2][0], landmarks[0][2][1]
  box_width, box_height = boxes[0][2]*0.8, boxes[0][3]*0.1
  width_margin, height_margin = 0.15*box_width, box_height*0.5
  start = (math.ceil(nose_x - width_margin), math.ceil(nose_y - height_margin*1.5))
  end = (math.ceil(nose_x + width_margin), math.ceil(nose_y + height_margin))
  cv2.rectangle(img, start, end, (0,0,0),-1)
  return img

def blackout_mouth(img, boxes, landmarks):
  left_mouth_x, left_mouth_y = landmarks[0][3][0], landmarks[0][3][1]
  right_mouth_x, right_mouth_y = landmarks[0][4][0], landmarks[0][4][1]
  box_width, box_height = boxes[0][2]*0.5, boxes[0][3]*0.15
  #defining width and height of black rectangle to be created
  width_margin, height_margin = 0.1*box_width, box_height*0.35
  ## old--->> width_margin, height_margin = abs(left_eye_x - right_eye_x)*0.25, 10
  #width_margin, height_margin = width*0.8, box_height*0.1
  start = (math.ceil(left_mouth_x - width_margin), math.ceil(left_mouth_y - height_margin))
  end = (math.ceil(right_mouth_x + width_margin), math.ceil(right_mouth_y + height_margin))
  cv2.rectangle(img, start, end, (0,0,0),-1)
  return img 

In [5]:
def blackout(img, boxes, probs, landmarks):
  if not probs[0]:     #incase no face detected by MTCNN
    return img

  if probs[0] < 0.95:  #in case MTCNN not sure about exact face landmarks, can be due to blurring
    return img

  m = randint(0, 3)  ##Random number
  
  if m==0:
    return blackout_eyes(img, boxes, landmarks)
  
  elif m==1:
    return blackout_nose(img, boxes, landmarks)

  else:
    return blackout_mouth(img, boxes, landmarks)

In [6]:
def mtcnn_params(img):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)
    boxes, probs, landmarks = mtcnn.detect(img, landmarks=True)
    return boxes, probs, landmarks

# 4 Preprocessing

## 4.1 Perform Augmentation with Albumentation

## 4.2 Experiment 